### CAMERA CALIBRATION



In [1]:
import numpy as np
import cv2
import scipy.io
import os
from numpy.linalg import norm
from matplotlib import pyplot as plt
from numpy.linalg import det
from numpy.linalg import inv
from scipy.linalg import rq
from numpy.linalg import svd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle

####This part of the code lets you use files from your Google Drive account
####You need not use this if you are not using Google Colaboratory
####Running code on Google Colab

(a) Make a folder on Google Drive, upload this file and all the data files provided to you into the same folder

(b) The last part of the URL will serve as the "folder_id" in the line below

If you are unsure, you can read further here : https://stackoverflow.com/questions/48376580/google-colab-how-to-read-data-from-my-google-drive

In [4]:
folder_id = ' ' # Enter the last part of the URL of the Google Drive folder here

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
    os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': '\'' + folder_id + "\' in parents"}).GetList()

for f in file_list:
    # 3. Create & download by id.
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = os.path.join(local_download_path, f['title'])
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
  
os.chdir(local_download_path)

ModuleNotFoundError: No module named 'google.colab'

####PART 1: Given corresponding 2d points in the image and 3d coordinates with known extrinsics, estimate the camera intrinsics of the form given in the CalibrationAssignment.docx

In [ ]:
# Code for Part 1 starts here

# Step 1: Load the data file pt_corres.mat
data_part1 = scipy.io.loadmat('pt_corres.mat')
cam_pts_3D = data_part1['cam_pts_3D']         # Load the 3d points
pts_2D = data_part1['pts_2D']                 # Load the corresponding 2d points

print pts_2D.shape
print cam_pts_3D.shape

# Step 2: Write your code here to compute the camera intrinsics 

#Obtain the transpose of two matrices
pts_2DT=np.transpose(pts_2D)
cam_pts_3DT=np.transpose(cam_pts_3D)

one=np.ones((28,1))

#Matrix AT is composed of three columns with the first column being the x values of the image points, 
#second column being the y values of the image points and the third column being zeros 

A = np.hstack((pts_2DT,one))
AT=np.transpose(A)

#Compose matrix BT such that 1st column is X/Z, 2nd column is Y/Z, 3rd column is one
x1=np.empty((28,1))
x2=np.empty((28,1))
for i in range(0,28):
  x1[i]=cam_pts_3DT[i][0]/cam_pts_3DT[i][2]
  x2[i]=cam_pts_3DT[i][1]/cam_pts_3DT[i][2]

B=np.hstack((x1,x2,one))
BT=np.transpose(B)
Binv = np.linalg.pinv(BT)
print("K", np.dot(AT,Binv))
# Code for Part 1 ends here

####PART 2: Given 2d points on the image and corresponding 3d points in the world-coordinate system, estimate both intrinsics and extrinsics. You need to fill in the code for the function calibrate() in calib_DLT.ipynb before running the cell below